### Demo Preparation Notebook

**Please Note**: This notebook and demo are NOT intended to be used as learning materials. To gain
a thorough understanding of the DataJoint workflow, please
see the [`tutorial`](./tutorial.ipynb) notebook.

In [ ]:
import os
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

In [ ]:
import datajoint as dj
import datetime
from workflow_miniscope.pipeline import subject, session, miniscope, Device

In [ ]:
subject.Subject.insert1(
    dict(
        subject='subject1',
        subject_birth_date='2023-01-01',
        sex='U',
    )
)

In [ ]:
Device.insert1(
    dict(device="UCLA Miniscope", 
         modality="miniscope"
        )
)

In [ ]:
session_key = dict(subject='subject1', 
                   session_datetime=datetime.datetime(2023, 5, 11, 12, 00, 00))

session.Session.insert1(session_key)

session.SessionDirectory.insert1(
    dict(
        session_key, 
        session_dir='subject1/session1'
    )
)

In [ ]:
miniscope.Recording.insert1(
    dict(
        session_key,
        recording_id=0,
        device="UCLA Miniscope",
        acq_software="Miniscope-DAQ-V4",
    )
)

In [ ]:
miniscope.RecordingInfo.populate(display_progress=True)

In [ ]:
caiman_params = dict(
    decay_time=0.4,
    pw_rigid=False,
    max_shifts=(5, 5),
    gSig_filt=(3, 3),
    strides=(48, 48),
    overlaps=(24, 24),
    max_deviation_rigid=3,
    border_nan="copy",
    method_init="corr_pnr",
    K=None,
    gSig=(3, 3),
    gSiz=(13, 13),
    merge_thr=0.7,
    p=1,
    tsub=2,
    ssub=1,
    rf=40,
    stride=20,
    only_init=True,
    nb=0,
    nb_patch=0,
    method_deconvolution="oasis",
    low_rank_background=None,
    update_background_components=True,
    min_corr=0.8,
    min_pnr=10,
    normalize_init=False,
    center_psf=True,
    ssub_B=2,
    ring_size_factor=1.4,
    del_duplicates=True,
    border_pix=0,
    min_SNR=3,
    rval_thr=0.85,
    use_cnn=False,
)

miniscope.ProcessingParamSet.insert_new_params(
    processing_method="caiman",
    paramset_id=0,
    params=caiman_params,
    paramset_desc='Default parameter set for CaImAn'
)

In [ ]:
miniscope.ProcessingTask.insert1(
    dict(
        session_key,
        recording_id=0,
        paramset_id=0,
        task_mode='load', # load or trigger
        processing_output_dir='subject1/session1/caiman',
    )
)

miniscope.Processing.populate(display_progress=True)

In [ ]:
miniscope.Curation.insert1(
    dict(
        session_key,
        recording_id=0,
        paramset_id=0,
        curation_id=0,
        curation_time=datetime.datetime(2023, 5, 11, 12, 00, 00),
        curation_output_dir='subject1/session1/caiman',
        manual_curation=False,
    )
)

miniscope.MotionCorrection.populate(display_progress=True)
miniscope.Segmentation.populate(display_progress=True)
miniscope.Fluorescence.populate(display_progress=True)
miniscope.Activity.populate(display_progress=True)